In [7]:
#!/usr/bin/env python
# coding: utf-8

import rosbag, os, matplotlib, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from qsrlib.qsrlib import QSRlib, QSRlib_Request_Message
from qsrlib_io.world_trace import Object_State, World_Trace
from qsrlib.qsrlib import QSR_QTC_BC_Simplified
import numpy as np
import pandas as pd
import datetime as dt

os.chdir("/home/loz/QTC_Trajectory_HMMs/from_bags/")

In [8]:
# In[2]:


lab_bags = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("~/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/")) for f in fn]
lab_bags[1].split("/")[-1][:2]

'hf'

In [9]:
lab_bags = [lab_bag for lab_bag in lab_bags if lab_bag.split("/")[-1][:2] == "hf"]
lab_bags

['/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/7/hf_study_2020-08-06-15-26-43.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/3/hf_study_2020-08-06-15-20-30.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/10/hf_study_2020-08-06-15-27-35.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/8/hf_study_2020-08-06-15-27-04.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/11/hf_study_2020-08-06-15-28-23.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/4/hf_study_2020-08-06-15-21-09.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/6/hf_study_2020-08-06-15-26-08.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/2/hf_study_2020-08-06-15-20-07.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/1/hf_study_2020-08-06-15-18-57.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/5/hf_study_2020

In [10]:
seqs = {"pbl": [], "pbr": [], "pcl": [], "pcr": [], "rotl": [], "rotr": [], "u": []}

for bag_path in lab_bags:
    bag = rosbag.Bag(bag_path)
    
    seq = []
    sit_code = "_".join(bag_path.split("/")[-3:-2])
#     print(sit_code)
    for topic, msg, t in bag.read_messages(topics=['/robot5/qsr/qtc_state']):
        if len(seq) == 0:
            seq.append(msg.data)
        elif seq[-1] != msg.data:
            seq.append(msg.data)
    
#     print("Complete seq: " + str(seq))
    for state_ix in range(len(seq)):
#         print(seq[:state_ix+1])
        seqs[sit_code].append(seq[:state_ix+1])

seqs

{'pbl': [['-,-,+,-'],
  ['-,-,+,-', '-,-,-,-'],
  ['-,-,+,-', '-,-,-,-', '+,+,-,-'],
  ['+,-,0,-'],
  ['+,-,0,-', '+,-,-,-'],
  ['+,-,0,-', '+,-,-,-', '-,-,-,-'],
  ['+,-,0,-', '+,-,-,-', '-,-,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,-,+,-'],
  ['-,-,-,-', '+,-,+,-', '+,+,-,-'],
  ['-,-,+,-'],
  ['-,-,+,-', '-,+,+,+'],
  ['-,-,+,-', '-,+,+,+', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,-,-', '+,+,-,-', '-,+,-,-'],
  ['-,-,-,-', '+,+,-,-', '-,+,-,-', '-,+,+,+'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,-,-', '+,+,-,-', '-,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,+,-'],
  ['-,-,+,-', '-,+,+,+'],
  ['-,-,+,-', '-,+,+,+', '-,-,-,-'],
  ['-,-,+,-', '-,+,+,+', '-,-,-,-', '+,+,-,-'],
  ['-,-,-,-']],
 'pbr': [['-,-,+,+'],
  ['-,-,+,+', '+,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+', '-,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+', '+,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+', '-,0,+,+'],
  ['-,-,+,0'],
  ['-,-,+,0', '-,-,

In [11]:
total_seqs = 0
for key in seqs.keys():
    total_seqs += len(seqs[key])
total_seqs

202

In [12]:
with open("hf_study_qtc_seqs.pickle", "w") as f:
    pickle.dump(seqs, f)

In [13]:
with open("hf_study_qtc_seqs.pickle", "r") as f:
    print(pickle.load(f))

{'pbr': [['-,-,+,+'], ['-,-,+,+', '+,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '+,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,0,+,+'], ['-,-,+,0'], ['-,-,+,0', '-,-,+,+'], ['-,-,+,0', '-,-,+,+', '-,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,+,+,+'], ['-,-,+,+', '-,+,+,+', '-,-,+,+'], ['-,-,+,+', '-,+,+,+', '-,-,+,+', '+,+,+,+'], ['-,-,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,+,+,+'], ['-,-,+,+', '-,+,+,+', '-,+,-,-'], ['-,-,+,+'], ['-,-,+,+', '-,+,-,+'], ['-,-,+,+', '-,+,-,+', '-,-,+,+'], ['-,-,+,+', '-,+,-,+', '-,-,+,+', '+,+,+,+'], ['-,-,+,0'], ['-,-,+,0', '-,-,+,+'], ['-,-,+,0', '-,-,+,+', '-,+,+,+']], 'pcr': [['-,-,+,-'], ['-,-,+,-', '+,-,+,-'], ['-,-,+,-'], ['-,-,+,-', '0,-,+,-'], ['-,-,+,-'], ['-,-,+,-', '+,-,+,-'], ['-,-,+,-', '+,-,+,-', '+,-,+,+'], ['-,-,+,-'], ['-,-,+,-', '-,-,+,+'], ['-,-,+,-'], ['-,-,+,-', '+,-,+,0'], ['-,-,+,-'], ['-,-,+,-', '+,-,+,-'], ['-,-,+,-', '+,-,+,-', '+,-,+,+'], ['-,-,+,-'], ['-,-,+,-'], ['-,-,+,-', '+,+,+,+'], ['-,-,+,-'], ['-,-,+,-', '+